# Download hydrology results from BigQuery
---

## Introduction
This notebook provides an example for downloading hydrology modeling results from the Nature Conservancy's Stormwater Heatmap. 
Access to the `tnc-data-v1` project in Google Cloud Platform is required. 

For more details and instructions see the [documentation on the stormwater heatmap website](https://www.stormwaterheatmap.org/docs/timeseries). 

---




## Variables 

### Grid ID

`grid_id` refers to the WRF precipitation grid id for the location of interest. 

### HRU 
`hru` refers to the [hydrologic response unit](https://www.stormwaterheatmap.org/docs/Data%20Layers/hydrologic_response_units) of interest. 

`hru` contains an three-digit encoding as described below: 

- First digit: Hydrologic Soil Group Number (0 = A/B, 1 = C, 2 = Saturated)
- Second digit: Land cover (0=Forest, 1=Pasture, 2=Lawn, 5=Impervious)
- Third Digit: Slope (0=Flat, 1=Mod, 2=Steep)

### Flow Path 

`flow_path` refers to the hspf flow path for which results are calculated. 

Available flow paths are: 
- `suro` - Surface Runoff 
- `ifwo` - Interflow
- `agwo` - outflow to groundwater

### Datetime

`Datetime` is the timestamp for the hourly simulation result. 




## Tables 
There are two options for accessing result tables: 
1. Grid and flowpath specific tables
2. Single table for all grids 

### Grid and flowpath specific tables.
To reduce query costs, this option should be used when querying a single grid or single flow path. Results have the following schema: `tnc-data-v1.{grid_id}.{flowpath}` 
```sql
SELECT
  datetime,
  hru
FROM
  tnc-DATA-v1.{grid_id}.{flowpath}
ORDER BY
  datetime


```

### Grid and flowpath specific tables.
A single table is available that holds all results. It is found at ```tnc-data-v1.hydrology.gfdl``` This table also includes useful columns such as `year`, `month`, and `simulation_day`

You can query this table for flow path and hru results. An example query is below: 

```sql 

SELECT
  datetime,
  SUM(hru250)
FROM
  tnc-data-v1.hydrology.gfdl
WHERE
  comp = 'suro'
  OR comp = 'agwo'
  and year between 1970 and 2000 
GROUP BY
  datetime
ORDER BY
  datetime
```

# Code

---


In [9]:
## Install client libraries if needed
#!pip install --upgrade pandas-gbq 

In [3]:
#import library
import pandas_gbq
#import tqdm - optional if you want to see a progress bar
#set project id
project_id = 'tacoma-app'


## Parameters 

In [4]:
table = "`tnc-data-v1.hydrology.gfdl`" #table where results are 
hru = "hru000" #hru code 
year_0 = 1970 #beginning year 
year_n = 1999 #ending year 
comps = '("suro", "ifwo")' #components 
    #- available options are:  
        #suro (surface runoff), 
        #ifwo (interflow), and 
        #agwo (groundwater outflow) - generally not used for stormwater results
grid = "'ID15_V5'" #WRF grid id 

In [5]:
sql = """
SELECT
  Datetime,
  SUM({}) AS mm_per_hour
FROM
  {}
WHERE
  year BETWEEN {}
  AND {}
  AND comp in {}
  AND grid = {}
GROUP BY
  Datetime
ORDER BY
  Datetime
""".format(hru,table,year_0,year_n,comps,grid)

Read the table from BigQuery. If not already authenticated, this command will trigger an authentication to Google Cloud Platform in a web browser.  

In [6]:
pandas_gbq.read_gbq(sql,project_id=project_id)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=tYCe2WeSpr493HPxB9QT0q6sZIJXz3&access_type=offline


Unable to create credentials directory.
/opt/miniconda3/envs/stormpiper/lib/python3.8/site-packages/google/cloud/bigquery/table.py:1939: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,Datetime,mm_per_hour
0,1970-01-02 00:00:00+00:00,0.0
1,1970-01-02 01:00:00+00:00,0.0
2,1970-01-02 02:00:00+00:00,0.0
3,1970-01-02 03:00:00+00:00,0.0
4,1970-01-02 04:00:00+00:00,0.0
...,...,...
262939,1999-12-31 19:00:00+00:00,0.0
262940,1999-12-31 20:00:00+00:00,0.0
262941,1999-12-31 21:00:00+00:00,0.0
262942,1999-12-31 22:00:00+00:00,0.0


View the data

<bound method NDFrame.head of                          datetime  hru250
0       1970-01-02 00:00:00+00:00     0.0
1       1970-01-02 01:00:00+00:00     0.0
2       1970-01-02 02:00:00+00:00     0.0
3       1970-01-02 03:00:00+00:00     0.0
4       1970-01-02 04:00:00+00:00     0.0
...                           ...     ...
1139520 2099-12-30 20:00:00+00:00     0.0
1139521 2099-12-30 21:00:00+00:00     0.0
1139522 2099-12-30 22:00:00+00:00     0.0
1139523 2099-12-30 23:00:00+00:00     0.0
1139524 2099-12-31 00:00:00+00:00     0.0

[1139525 rows x 2 columns]>